<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Twitter
`Fecha de entrega: Mayo 20, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [20]:
import pandas as pd
import tweepy

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [21]:
# Leer las llaves
keys = pd.read_csv("G:/Mi unidad/NANI/_ESTUDIOS/_Especialización/Semestre 2/NLP/Taller 11/Mis_claves.csv", header=None)
keys = dict(zip(keys[0],keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [22]:
# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [23]:
tweets = tweepy.Cursor(api.search,
                       q=['paro nacional -RT'],
                       lang='es',
                       geocode='4.7110,-74.0721,200km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(100)

In [24]:
# Objeto generador
t = [tweet for tweet in tweets]

In [25]:
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)
    
    
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweets,date,city,user_location,id,link
0,#Especiales Colombia 🇨🇴 Despierta: Crónicas de...,2021-05-20 22:01:26,Bogotá,"Bogotá, D.C., Colombia",1395499935601147920,https://twitter.com/i/web/stuatus/139549993560...
1,Esta tarde habrá PMU abierto en el Portal Amér...,2021-05-20 22:00:26,Bogotá,"Bogotá, Colombia",1395499682806259715,https://twitter.com/i/web/stuatus/139549968280...
2,#Editorial Entre las muchas afectaciones que h...,2021-05-20 22:00:13,Bogotá,Santiago de Cali,1395499630520152064,https://twitter.com/i/web/stuatus/139549963052...
3,Pérdidas por más de $18.000 millones deja paro...,2021-05-20 22:00:13,Bogotá,Colombia,1395499627697291268,https://twitter.com/i/web/stuatus/139549962769...
4,El Gobierno insiste en que ha resuelto las inq...,2021-05-20 22:00:00,Bogotá,Medellín,1395499577122299904,https://twitter.com/i/web/stuatus/139549957712...


In [26]:
data.to_csv("archivo.csv", index=False)

###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [27]:
data['hora'] = data['date'].dt.floor('T').dt.time
temp = pd.DataFrame(data.hora.value_counts()).reset_index()
temp.columns = ['hora', 'cnt']
temp = temp.sort_values('hora')
temp

,hora,cnt
7,21:00:00,3
9,21:01:00,3
40,21:02:00,1
6,21:03:00,3
2,21:04:00,4
22,21:05:00,2
49,21:06:00,1
36,21:07:00,1
37,21:08:00,1
43,21:10:00,1


In [33]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [34]:
trace = go.Scatter(x=temp.hora.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.hora.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Horas',
                   line={'color': 'blue'})

layout = go.Layout(title="Número de tuits por hora")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)